# Introducción


## Objetivo

Utilizar Llama 2.0, Langchain y ChromaDB para crear un sistema de Generación con Recuperación Mejorada (RAG). Esto nos permitirá hacer preguntas sobre nuestros documentos (que no se incluyeron en los datos de entrenamiento), sin necesidad de ajustar finamente el Modelo de Lenguaje Grande (LLM, por sus siglas en inglés).
Cuando se utiliza RAG, si se plantea una pregunta, primero se realiza un paso de recuperación para obtener documentos relevantes de una base de datos especial, una base de datos vectorial donde se indexaron estos documentos.

## Definiciones

* LLM - Modelo de Lenguaje Grande (Large Language Model)
* Llama 2.0 - LLM de Meta
* Langchain - un marco diseñado para simplificar la creación de aplicaciones utilizando LLM
* Base de datos vectorial - una base de datos que organiza datos a través de vectores de alta dimensión
* ChromaDB - base de datos vectorial
* RAG - Generación con Recuperación Mejorada (consulte más detalles sobre RAG a continuación)

## Detalles del modelo

* **Modelo**: Llama 2
* **Variante**: 7b-chat-hf (7b: 7 mil millones de dimensiones, hf: compilación de HuggingFace)
* **Versión**: V1
* **Framework**: PyTorch

El modelo LlaMA 2 está preentrenado y ajustado con 2 billones de tokens y de 7 a 70 mil millones de parámetros, lo que lo convierte en uno de los modelos de código abierto más potentes. Es una mejora significativa con respecto al modelo LlaMA 1.


## ¿Qué es un sistema de Generación con Recuperación Mejorada (RAG)?

Los Modelos de Lenguaje Grande (LLM) han demostrado su capacidad para comprender el contexto y proporcionar respuestas precisas a diversas tareas de Procesamiento de Lenguaje Natural (NLP), incluyendo la resumen, preguntas y respuestas, cuando se les solicita. Si bien son capaces de proporcionar respuestas muy buenas a preguntas sobre información con la que fueron entrenados, tienden a alucinar cuando el tema trata sobre información que "no saben", es decir, no estaba incluida en sus datos de entrenamiento. La Generación con Recuperación Mejorada combina recursos externos con LLM. Por lo tanto, los dos componentes principales de un sistema RAG son un recuperador y un generador.

La parte del recuperador se puede describir como un sistema que es capaz de codificar nuestros datos para que se puedan recuperar fácilmente las partes relevantes al consultarlos. La codificación se realiza utilizando incrustaciones de texto, es decir, un modelo entrenado para crear una representación vectorial de la información. La mejor opción para implementar un recuperador es una base de datos vectoriales. Como bases de datos vectoriales, existen múltiples opciones, tanto productos de código abierto como comerciales. Algunos ejemplos son ChromaDB, Mevius, FAISS, Pinecone, Weaviate. Nuestra opción en este cuaderno será una instancia local de ChromaDB (persistente).

Para la parte del generador, la opción más obvia es un LLM. En este cuaderno utilizaremos un modelo LLaMA v2 cuantificado, de la colección de Modelos de Kaggle.

La orquestación del recuperador y el generador se realizará utilizando Langchain. Una función especializada de Langchain nos permite crear el recuperador-generador en una sola línea de código.


# Installations, imports, utils

In [1]:
!pip install transformers==4.33.0 accelerate==0.22.0 einops==0.6.1 langchain==0.0.300 xformers==0.0.21 \
bitsandbytes==0.41.1 sentence_transformers==2.2.2 chromadb==0.4.12

In [2]:
pip install docx2txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
from time import time
from langchain.llms import HuggingFacePipeline
from langchain import document_loaders 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.prompts import PromptTemplate
from bs4 import BeautifulSoup as Soup

# Inicializar modelo, tokenizador, y canal de consultas.

Define el modelo, el dispositivo y la configuración de `bitsandbytes`.

In [4]:
model_id = '/kaggle/input/llama-2/pytorch/7b-chat-hf/1'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

Preparar el modelo y el tokenizador.

In [5]:
time_1 = time()

model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


Prepare model, tokenizer: 199.118 sec.


Definir el query pipeline

In [6]:
time_1 = time()

query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",)

time_2 = time()
print(f"Prepare pipeline: {round(time_2-time_1, 3)} sec.")

Prepare pipeline: 1.992 sec.


Definir una función para testear el pipeline

In [7]:
def test_model(tokenizer, pipeline, prompt_to_test):
    """
    Perform a query
    print the result
    Args:
        tokenizer: the tokenizer
        pipeline: the pipeline
        prompt_to_test: the prompt
    Returns
        None
    """
    # adapted from https://huggingface.co/blog/llama2#using-transformers
    time_1 = time()
    sequences = pipeline(
        prompt_to_test,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=200,)
    time_2 = time()
    print(f"Test inference: {round(time_2-time_1, 3)} sec.")
    for seq in sequences:
        print(f"Result: {seq['generated_text']}")

## Testear la query pipeline

Testeamos el pipeline con una query sobre...

In [8]:
test_model(tokenizer,
           query_pipeline,
           "Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


Test inference: 11.79 sec.
Result: Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.
The State of the Union address is an annual speech delivered by the President of the United States to Congress, in which the President reviews the current state of the nation and outlines his or her legislative agenda for the upcoming year.
State of the Union address is a speech given by the President to Congress, it's an annual event where the President reviews the current state of the nation and presents his legislative agenda for the upcoming year


In [9]:
test_model(tokenizer,
           query_pipeline,
           "Explica que es el discurso sobre el estado de la nación en EE.UU. Hazlo en menos de 100 palabras.")

Test inference: 13.314 sec.
Result: Explica que es el discurso sobre el estado de la nación en EE.UU. Hazlo en menos de 100 palabras.

El discurso sobre el estado de la nación en EE.UU. es un mensaje anual que el presidente de los Estados Unidos lee a Congress, en el que describe el estado actual de la nación y los desafíos que enfrenta.

Este discurso es una oportunidad para el líder de la nación evaluar el progreso realizado en diferentes áreas, como la económica, la social y la internacional, y para presentar propuestas para mejorar el estado de la nación. Además, el discurso es un momento crítico para establecer la agenda política del gobierno durante el próximo año.


# Retrieval Augmented Generation

## Comprobar el modelo con HuggingFace pipeline


Testeamos el modelo con HF pipeline, usando una query sobre.

In [10]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Please explain what is the State of the Union address. Give just a definition. Keep it in 100 words.")

'\nThe State of the Union address is an annual speech given by the President of the United States to a joint session of Congress, in which the President reports on the current state of the union and outlines their legislative agenda for the upcoming year.'

In [11]:
llm = HuggingFacePipeline(pipeline=query_pipeline)
# checking again that everything is working fine
llm(prompt="Explica que es el discurso sobre el estado de la nación en EE.UU. Hazlo en menos de 100 palabras.")

'\n\nEl discurso sobre el estado de la nación en EE.UU. es un mensaje anual que el presidente de los Estados Unidos lee ante el Congreso y el público en general. El discurso se centra en la situación actual de la nación, incluyendo el estado de la economía, la seguridad nacional, la educación y la política exterior. El presidente utiliza este momento para informar a los ciudadanos sobre los logros y desafíos de la nación y para establecer objetivos y metas para el futuro.'

## Data Ingestion usando Loaders

Vamos a adquirir información de distintas fuentes y a juntarla.

In [58]:
# Adquirir datos de distintas fuentes
loaders = [
    #document_loaders.Docx2txtLoader("/kaggle/input/tfg-datasetstest/Listado Preguntas-Respuestas - ONLINE.docx"),
    #document_loaders.Docx2txtLoader("/kaggle/input/tfg-datasetstest/Listado Preguntas-Respuestas - PRESENCIAL.docx"),
    document_loaders.TextLoader("/kaggle/input/tfg-datasetstest/Listado Preguntas-Respuestas - ONLINE.txt", encoding='utf8'),
    #document_loaders.PyPDFLoader("/kaggle/input/tfg-datasetstest/2-entrega_proyecto_fin_de_grado_vena.pdf"),
    #document_loaders.PyPDFLoader("/kaggle/input/tfg-datasetstest/3-vena-modelo_informe_tutor_academico_modificado.pdf"),
    document_loaders.PyPDFLoader("/kaggle/input/tfg-datasetstest/reglamentp_tfg-tfm_aprob._08-06-2022.pdf"),
    #document_loaders.GitHubIssuesLoader(repo="langchain-ai/langchain",access_token=ACCESS_TOKEN, creator="UmerHA",)
    
    #document_loaders.recursive_url_loader.RecursiveUrlLoader(url="https://clopezno.github.io/tfg_gii_online/HistoricoSist.html", 
    #                                                         max_depth=2, extractor=lambda x: Soup(x, "html.parser").text),
    
    #document_loaders.WebBaseLoader("https://www.ubu.es/grado-en-ingenieria-informatica/informacion-basica/trabajo-fin-de-grado")   
]

documents = []

for loader in loaders:
    documents.extend(loader.load())

## Trocear los datos en chunks

Dividimos los datos en chunks utilizando un separador de texto de caracteres recursivo.

In [59]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
all_splits = text_splitter.split_documents(documents)

## Crear Embeddings y guardarlos en una BD Vectorial

Create the embeddings using Sentence Transformer and HuggingFace embeddings.

In [60]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

Inicializar ChromaDB con los chunks, embeddings y con persistencia local.

In [61]:
persist_directory="chroma_db"
!rm -rf ./chroma_db  # remove old database files if any
vectordb = Chroma.from_documents(
    documents=all_splits, 
    embedding=embeddings, 
    persist_directory=persist_directory)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

## Custom Prompt

Crear un Propmpt customizado para forzar cotexto e idioma.

In [83]:
prompt_template = """<s>[INST] You are given the context after <<CONTEXT>> and a question after <<QUESTION>>.

You are a chatbot for the preparation of Thesis project(TFG). Answer in the same language {question}.

<<QUESTION>>{question}\n<<CONTEXT>>{context} [/INST]"""
PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

## Inicializar chain

In [84]:
# Retrieve more documents with higher diversity- useful if your dataset has many similar documents
#retriever = vectordb.as_retriever(search_kwargs = {"k": 5, "search_type" : "similarity"})
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    chain_type_kwargs={"prompt": PROMPT},
    verbose=False
)

## Test Retrieval-Augmented Generation 


Definimos una función de prueba que ejecutará la consulta y medirá el tiempo.

In [85]:
def test_rag(qa, query):
    print(f"Query: {query}\n")
    time_1 = time()
    result = qa.run(query)
    time_2 = time()
    print(f"Inference time: {round(time_2-time_1, 3)} sec.")
    print("\nResult: ", result)

Comprobemos algunas consultas.

In [86]:
query = "¿Se pueden adjuntar videos en el depósito del TFG??"
test_rag(qa, query)

Query: ¿Se pueden adjuntar videos en el depósito del TFG??



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inference time: 30.348 sec.

Result:    ¡Hola! ¡Claro que sí! Puedes adjuntar videos en el depósito del TFG. En la convocatoria de la asignatura, se habilita una tarea en UBUVirtual donde puedes subir los ficheros necesarios, incluyendo videos, para la evaluación del TFG.

Es importante mencionar que los videos debe estar colgados en YouTube y proporcionar los enlaces en el depósito del TFG. Esto se debe a que la plataforma de UBUVirtual no permite la subida de ficheros directamente desde el ordenador del estudiante.

Recuerda que la presentación y defensa del TFG pueden ser realizadas en modalidad on-line, por lo que es importante estar preparado para presentar y defender tu trabajo de manera efectiva a través de la videoconferencia.

Si tienes alguna otra pregunta o necesitas más información, no dudes en preguntar. Estoy aquí para ayudarte en lo que pueda. ¡Buena suerte con tu TFG!


In [87]:
query = "¿Puedo cambiar de tutor o tema?"
test_rag(qa, query)

Query: ¿Puedo cambiar de tutor o tema?



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inference time: 50.025 sec.

Result:    ¡Hola! ¡Claro que sí! Puedes cambiar de tutor o tema en tu TFG. Para hacerlo, deberás solicitarlo al Tribunal por escrito, en un plazo máximo de un mes desde la fecha de la publicación de la asignación. El Tribunal estará obligado a contestar al alumno por escrito en un plazo máximo de siete días hábiles, en el que se motivará la resolución del TFG/TFM a la petición del alumno.

En cuanto a la plantilla para la memoria, existente un modelo de plantilla en LaTeX en el siguiente enlace: <https://github.com/ubutfgm/plantillaLatex>.

También puedes elegir tutor dependiendo de la modalidad de proyecto. En la modalidad C, el estudiante tiene que acordar previamente su propuesta con un tutor que acepte su temática. En la modalidad A, las propuestas de TFG concretas se publican y los tutores pueden indicar su interés en trabajar con ciertas propuestas. En la modalidad B, las empresas ofrecen propuestas de TFG y el alumno puede elegir el tutor que desee t

In [88]:
query = "¿Que pasaria si no me gusta el tema o tutor que he escogido?"
test_rag(qa, query)

Query: ¿Que pasaria si no me gusta el tema o tutor que he escogido?



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inference time: 53.593 sec.

Result:    ¡Hola! Si no te gusta el tema o el tutor que has escogido para tu TFG, no te preocupes, es normal sentir dudas o deseos de cambiar en algún momento del proceso.

En cuanto a la elección del tutor, en la modalidad C de oferta de TFG por el alumno, es recomendable identificar los conceptos teóricos principales que se necesitan en el TFG y buscar a los profesores de las guías docentes de las asignaturas del Grado que te los enseñaron. Esto te ayudará a encontrar a un tutor que tenga experiencia y conocimientos en el área de estudio que te interesa.

Si no puedes elegir a tu tutor, en la modalidad A, puedes presentar una propuesta de TFG concreta y publicarla en el portal de UBUVirtual. En la modalidad B, las empresas ofrecen la oportunidad de trabajar con un tutor de la asignatura que acepten la propuesta de TFG.

Si deseas cambiar de tutor o tema, puedes solicitarlo al Tribunal por escrito, en un plazo máximo de un mes desde la fecha de la publicac

In [89]:
query = "¿Can you give examples of past TFG?"
test_rag(qa, query)

Query: ¿Can you give examples of past TFG?



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inference time: 44.185 sec.

Result:    Examples of past TFG (Thesis Project) include:

1. Development of a Machine Learning Model for Predicting Customer Churn in a Telecommunications Company: This TFG aimed to create a machine learning model that could predict customer churn in a telecommunications company, using data from various sources such as customer demographics, usage patterns, and feedback.
2. Design and Implementation of a Smart Home Automation System using IoT and Cloud Computing: This TFG focused on the design and implementation of a smart home automation system using Internet of Things (IoT) devices and cloud computing. The system was designed to provide a comfortable and efficient living space for the elderly and people with disabilities.
3. Development of a Mobile Application for Monitoring and Managing Chronic Diseases: This TFG developed a mobile application that allowed users to monitor and manage chronic diseases such as diabetes, hypertension, and asthma. The appli

## Fuentes del documento

Verifiquemos las fuentes de documentos para la última consulta ejecutada.

In [57]:
docs = vectordb.similarity_search(query)
print(f"Query: {query}")
print(f"Retrieved documents: {len(docs)}")
for doc in docs:
    doc_details = doc.to_json()['kwargs']
    print("Source: ", doc_details['metadata']['source'])
    print("Text: ", doc_details['page_content'], "\n")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Query: ¿Can you give examples of past TFG?
Retrieved documents: 4
Source:  /kaggle/input/tfg-datasetstest/Listado Preguntas-Respuestas - ONLINE.docx
Text:  que_es_tfg

¿Qué es el TFG?

Un TFG es aplicación autónoma e integral de algunos conceptos aprendidos en las distintas asignaturas del Grado. El estudiante actúa como gestor del proceso de para resolver los problemas planteados en TFG. Todo TFG tiene un tutor o tutora del Grado asignado.

que_es_un_anexo

¿Qué es un anexo?

Un anexo es un documento explicativo de las características técnicas del TFG a definir por consenso con el tutor/a. Se recomienda seguir la estructura disponible en el siguiente enlace:

https://github.com/ubutfgm/plantillaLatex

quienes_estaran_ exposicion

¿Quiénes estarán en la exposición?

La presentación/exposición de los TFG es un acto social y público donde exponer los trabajos a los asistentes que se lo soliciten al tribunal. 

Source:  /kaggle/input/tfg-datasetstest/Listado Preguntas-Respuestas - ONLINE.

# Conclusión

Utilizamos Langchain, ChromaDB y Llama 2 como un Large Language Model (LLM) para construir una solución con Retrieval-Augmented Generation. Para las pruebas, estábamos utilizando...


# Mas trabajos relacionados 

* https://www.kaggle.com/code/gpreda/test-llama-2-quantized-with-llama-cpp (quantizing LLama 2 model using llama.cpp)
* https://www.kaggle.com/code/gpreda/fast-test-of-llama-v2-pre-quantized-with-llama-cpp  (quantized Llamam 2 model using llama.cpp)  
* https://www.kaggle.com/code/gpreda/test-of-llama-2-quantized-with-llama-cpp-on-cpu (quantized model using llama.cpp - running on CPU)
+ https://www.kaggle.com/code/acorn8/q-a-with-llms-harry-potter-mistral-hf-api (CFG Class)


# References  

[1] Murtuza Kazmi, Using LLaMA 2.0, FAISS and LangChain for Question-Answering on Your Own Data, https://medium.com/@murtuza753/using-llama-2-0-faiss-and-langchain-for-question-answering-on-your-own-data-682241488476  

[2] Patrick Lewis, Ethan Perez, et. al., Retrieval-Augmented Generation for Knowledge-Intensive NLP Tasks, https://browse.arxiv.org/pdf/2005.11401.pdf 

[3] Minhajul Hoque, Retrieval Augmented Generation: Grounding AI Responses in Factual Data, https://medium.com/@minh.hoque/retrieval-augmented-generation-grounding-ai-responses-in-factual-data-b7855c059322  

[4] Fangrui Liu	, Discover the Performance Gain with Retrieval Augmented Generation, https://thenewstack.io/discover-the-performance-gain-with-retrieval-augmented-generation/

[5] Andrew, How to use Retrieval-Augmented Generation (RAG) with Llama 2, https://agi-sphere.com/retrieval-augmented-generation-llama2/   

[6] Yogendra Sisodia, Retrieval Augmented Generation Using Llama2 And Falcon, https://medium.com/@scholarly360/retrieval-augmented-generation-using-llama2-and-falcon-ed26c7b14670

[7] Using a Retriever in Langchain - https://python.langchain.com/docs/use_cases/question_answering/vector_db_qa